In [1]:
import xarray as xr
import numpy  as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shap
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import xgboost as xgb
import pickle
import sklearn

/glade/work/hpzhang/tools/anaconda3/envs/geocat/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [2]:
ff = pd.read_pickle("/glade/work/hpzhang/LCC_pred/data/data_monthly.pkl").dropna()
df  = ff.drop(columns=['lat', 'lon', 'lsm', 'T1000', 'T700', 'EIS', 'ECTEI','ELF','SST','AOD'])
df = df[df.index.year.isin(np.arange(2003,2019))]
len(df)

5350237

In [3]:
df.head()

,LCF,TH1000,TH850,TH700,RH1000,RH850,RH700,Q1000,Q850,Q700,U1000,U700,OMEGA500,OMEGA700,PWV,LH,SH,Tadv,dQ,LTS
time,,,,,,,,,,,,,,,,,,,,
2003-01-15,0.590822,273.145630,277.822327,286.950775,85.577278,84.145813,69.974464,0.003216,0.002045,0.001219,3.637893,6.459276,0.004265,0.005796,8.286732,-14.891875,-1.580903,0.898143,0.001997,13.805145
2003-01-15,0.590822,273.192474,277.799255,286.895844,85.665604,84.507431,69.732834,0.003221,0.002052,0.001191,3.618882,6.377827,0.002914,0.010119,8.276699,-13.840185,-0.886921,-0.020427,0.002030,13.703369
2003-01-15,0.556442,273.239288,277.803558,286.847046,85.945564,84.507431,69.224571,0.003238,0.002052,0.001163,3.703815,6.273440,0.001923,0.008408,8.276699,-13.721111,-0.987361,-0.006477,0.002075,13.607758
2003-01-15,0.556442,273.258575,277.851196,286.802826,86.460487,84.855713,69.139580,0.003261,0.002063,0.001153,3.733388,6.172708,0.005346,0.004445,8.293421,-12.907546,-0.686019,0.069008,0.002108,13.544250
2003-01-15,0.584355,273.276489,277.882935,286.755585,86.692123,85.037354,69.027931,0.003274,0.002071,0.001152,3.706906,6.052912,0.009849,0.000932,8.286732,-12.451134,-0.850394,-1.276983,0.002122,13.479095


In [4]:
#data = df.drop(columns=['Tadv', 'dQ', 'LTS'])
data = df
index_train = data.index.year.isin(np.arange(2003,2015))
index_test  = data.index.year.isin(np.arange(2015,2019))
data_train = data[index_train]
data_test  = data[index_test]

X_train, y_train = data_train.iloc[:,1:], data_train.iloc[:,0]
X_test, y_test   = data_test.iloc[:,1:], data_test.iloc[:,0]

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [5]:
'''
from sklearn.model_selection import RandomizedSearchCV

params = { 'max_depth': [5, 6, 10],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [200, 500, 1000]}
xgbr = xgb.XGBRegressor(tree_method='gpu_hist', 
                        eval_metric=mean_squared_error,
                        gpu_id=0, seed = 1001, n_jobs=-1)
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=50,
                         verbose=3,
                         cv=3)
clf.fit(X_train_scaled, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))
print("Lowest MSE: ", (-clf.best_score_))
'''

'\nfrom sklearn.model_selection import RandomizedSearchCV\n\nparams = { \'max_depth\': [5, 6, 10],\n           \'learning_rate\': [0.01, 0.1, 0.2, 0.3],\n           \'subsample\': np.arange(0.5, 1.0, 0.1),\n           \'colsample_bytree\': np.arange(0.4, 1.0, 0.1),\n           \'colsample_bylevel\': np.arange(0.4, 1.0, 0.1),\n           \'n_estimators\': [200, 500, 1000]}\nxgbr = xgb.XGBRegressor(tree_method=\'gpu_hist\', \n                        eval_metric=mean_squared_error,\n                        gpu_id=0, seed = 1001, n_jobs=-1)\nclf = RandomizedSearchCV(estimator=xgbr,\n                         param_distributions=params,\n                         scoring=\'neg_mean_squared_error\',\n                         n_iter=50,\n                         verbose=3,\n                         cv=3)\nclf.fit(X_train_scaled, y_train)\nprint("Best parameters:", clf.best_params_)\nprint("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))\nprint("Lowest MSE: ", (-clf.best_score_))\n'

In [6]:
'''
from sklearn.model_selection import GridSearchCV

params = { 'max_depth': [5, 6, 7, 8, 9, 10],
#           'learning_rate': [0.01, 0.1, 0.2, 0.3],
#           'subsample': np.arange(0.5, 1.0, 0.1),
#           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
#           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}
xgbr = xgb.XGBRegressor(tree_method='gpu_hist', 
                        eval_metric=mean_squared_error,
                        gpu_id=0, seed = 1001, n_jobs=-1)

clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params,
                   scoring='neg_mean_squared_error', 
                   verbose=3,
                   cv=5)
clf.fit(X_train_scaled, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))
print("Lowest MSE: ", (-clf.best_score_))
'''

'\nfrom sklearn.model_selection import GridSearchCV\n\nparams = { \'max_depth\': [5, 6, 7, 8, 9, 10],\n#           \'learning_rate\': [0.01, 0.1, 0.2, 0.3],\n#           \'subsample\': np.arange(0.5, 1.0, 0.1),\n#           \'colsample_bytree\': np.arange(0.4, 1.0, 0.1),\n#           \'colsample_bylevel\': np.arange(0.4, 1.0, 0.1),\n           \'n_estimators\': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}\nxgbr = xgb.XGBRegressor(tree_method=\'gpu_hist\', \n                        eval_metric=mean_squared_error,\n                        gpu_id=0, seed = 1001, n_jobs=-1)\n\nclf = GridSearchCV(estimator=xgbr, \n                   param_grid=params,\n                   scoring=\'neg_mean_squared_error\', \n                   verbose=3,\n                   cv=5)\nclf.fit(X_train_scaled, y_train)\nprint("Best parameters:", clf.best_params_)\nprint("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))\nprint("Lowest MSE: ", (-clf.best_score_))\n'

In [7]:
# create a binary pickle file 
#f = open("cv_results.pkl","wb")
#pickle.dump(clf.cv_results_, f)
#f.close()

#f = open("/glade/u/home/hpzhang/work/LCC_pred/optimize/cv_results.pkl",'rb')
#cv_res = pickle.load(f)
#f.close()

In [8]:
#rmse=(-cv_res["mean_test_score"])**(1/2.0)
#std=cv_res["std_test_score"]
#plt_dat=pd.DataFrame(rmse.reshape(-1,10), index=np.arange(5,11), columns=np.arange(100,1001,100))

In [9]:
#f, ax = plt.subplots(figsize=(20, 10))
#ax = sns.heatmap(plt_dat, linewidth=0.5,
#                cmap="YlGnBu_r",
#                annot=True,
#                fmt='5.5f')
#plt.show()

In [10]:
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

pbounds = {
    'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'max_depth': (3,7),
    'subsample': (0.5, 1.0),  
    'colsample_bytree': (0.5, 1.0), 
    'colsample_bylevel': (0.5, 1.0)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        max_depth,
                        subsample,
                        colsample_bytree,
                        colsample_bylevel):

    n_estimators = int(n_estimators)
    max_depth = int(max_depth)

    clf = xgb.XGBRegressor(tree_method='gpu_hist', 
                        eval_metric=mean_squared_error,
                        gpu_id=0, seed = 1001, n_jobs=-1,
                        learning_rate=learning_rate,
                        n_estimators=n_estimators,
                        max_depth=max_depth,
                        subsample=subsample,
                        colsample_bytree=colsample_bytree,
                        colsample_bylevel=colsample_bylevel)

    return np.mean(cross_val_score(clf, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error'))

optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1001,
)

optimizer.maximize(n_iter=15, init_points=3)


|   iter    |  target   | colsam... | colsam... | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -0.004008 |  0.6531   |  0.6325   |  0.2041   |  4.722    |  120.8    |  0.5979   |
|  2        | -0.003341 |  0.6764   |  0.6116   |  0.6174   |  5.322    |  868.2    |  0.5206   |
|  3        | -0.003511 |  0.7441   |  0.9604   |  0.118    |  4.644    |  523.1    |  0.5636   |
|  4        | -0.003064 |  0.9179   |  0.6096   |  0.2472   |  6.552    |  868.2    |  0.8313   |
|  5        | -0.003025 |  0.8997   |  0.8853   |  0.1618   |  6.971    |  870.0    |  0.9239   |
|  6        | -0.003059 |  0.7617   |  0.9691   |  0.2777   |  6.739    |  874.5    |  0.5925   |
|  7        | -0.004326 |  0.5772   |  0.5177   |  0.9992   |  6.321    |  879.1    |  0.5998   |
|  8        | -0.003372 |  0.8052   |  0.6569   |  0.6109   |  3.7      |  873.1    |  0.7883   |
|  9        | -0.003

In [11]:
optimizer.max['params']

{'colsample_bylevel': 0.8996518880817399,
 'colsample_bytree': 0.8853387899201755,
 'learning_rate': 0.16184400108442132,
 'max_depth': 6.970737464733581,
 'n_estimators': 869.9900903438686,
 'subsample': 0.9239287537315828}

In [12]:
#sklearn.metrics.get_scorer_names()

In [13]:
#for i, res in enumerate(optimizer.res):
#    print("Iteration {}: \n\t{}".format(i, res))